#### Stuff Document Chain Summarization

In [2]:
from langchain_community.document_loaders import PyPDFLoader

#loading PDF
loader = PyPDFLoader("apjspeech.pdf")
#Splitting into docs(each page)
docs =  loader.load_and_split()
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [10]:
len(docs)

7

In [4]:
#Creating Prompt template
from langchain import PromptTemplate
from langchain.chains import LLMChain
template = """
Write a concise and short summary of the following speech,
Speech:{text}
"""
prompt = PromptTemplate(input_variables=['text'],
                             template=template)

In [5]:
from langchain.chains.summarize import load_summarize_chain
#load summarize chain is going to combine the entire documents


In [6]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")

In [7]:
#creating chain
#Inside chain we are going to use 3 chain types
# 1. Stuff Documnet
# 2. Map Reduce
# 3. Refine
chain = load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)
chain.run(docs)

C:\Users\mahendhar Baira\AppData\Local\Temp\ipykernel_3344\4079981922.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(docs)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise and short summary of the following speech,
Speech:A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online inte

'President A.P.J. Abdul Kalam\'s departing speech emphasizes India\'s potential to become a developed nation by 2020. He highlights key messages gleaned from his interactions with people across various sectors:\n\n**Empowering the Youth:** Kalam stresses the importance of youth aspiration and their role in achieving India\'s development goals.\n\n**Rural Development:** He advocates for empowering villages, improving connectivity, and leveraging rural core competence for economic growth.\n\n**Agricultural Growth:** "Seed to Food" is presented as a crucial strategy for doubling agricultural production and improving farmer livelihoods.\n\n**Overcoming Challenges:** Kalam celebrates resilience and courage in the face of calamities, citing examples like the earthquake in Jammu & Kashmir and the Tsunami in Car Nicobar Islands.\n\n**Connectivity for Transformation:** He emphasizes the transformative power of technology, showcasing the Pan African e-Network project as an example.\n\n**National

#### Map Reduce ---> To summarize Large Documents

In [ ]:
#For splitting Text
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000,chunk_overlap = 100).split_documents(docs)

text_splitter

# we get many more chunks


[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [9]:
len(text_splitter)

13

In [11]:
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nWrite a concise and short summary of the following speech,\nSpeech:{text}\n')

In [13]:
chunks_prompt = """ Please summarize the below speech:
Speech:'{text}' 
Summary: 
"""
map_prompt_template = PromptTemplate(input_variables=['text'],
                             template=chunks_prompt)
map_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template=" Please summarize the below speech:\nSpeech:'{text}' \nSummary: \n")

In [14]:
final_prompt = """ 
Provide the final summary of the entire speech with  these important points,
Add a Motivation Title too
Speech:{text}"""

final_prompt_template = PromptTemplate(input_variables=['text'],
                             template=final_prompt)

final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template=' \nProvide the final summary of the entire speech with  these important points,\nAdd a Motivation Title too\nSpeech:{text}')

In [15]:
#Creating a summary chain
map_summary_chain = load_summarize_chain(llm,
                                         chain_type='map_reduce',
                                         map_prompt=map_prompt_template,
                                         combine_prompt = final_prompt_template,
                                         verbose=True)


output = map_summary_chain.run(text_splitter)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Please summarize the below speech:
Speech:'A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have ma

m:\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
m:\Langchain\venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mahendhar Baira\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.m



> Entering new LLMChain chain...
Prompt after formatting:
 
Provide the final summary of the entire speech with  these important points,
Add a Motivation Title too
Speech:In his departing speech, A.P.J. Abdul Kalam reflects on his five years as President of India, expressing gratitude for the experiences and interactions he had with people from all walks of life.  

He highlights ten key messages that emerged from these interactions:

1. **Accelerate Development:** Driven by the aspirations of the youth.
2. **Empower Villages:** Focusing on rural development.
3. **Mobilize Rural Competence:** Utilizing rural skills for economic competitiveness.
4. **Seed to Food:** Strengthening the agricultural sector.
5. **Defeat Problems and Succeed:** Overcoming challenges through determination.
6. **Partnership Over Problems:** Collaborating to solve issues.
7. **Courage in Calamities:** Facing adversity with resilience.
8. **Connectivity for Transformation:** Leveraging technology and communica

'##  Building a Developed India: A Vision for the Future \n\nThis speech is a powerful reflection on the journey of A.P.J. Abdul Kalam as President of India, encapsulating his vision for a developed and prosperous nation by 2020. \n\n**Key Takeaways:**\n\n* **Empowering Rural India:** Kalam emphasizes the need to uplift 600,000 villages through infrastructure development, connectivity, and the Periyar PURA initiative, which empowers rural communities through education, healthcare, and economic opportunities.\n* **Harnessing the Power of Youth:**  He identifies India\'s 540 million youth as the driving force behind this development, highlighting their aspirations for a developed, safe, and prosperous India. \n* **Agricultural Transformation:**  Kalam stresses the importance of "Seed to Food" approach to agriculture, advocating for a holistic approach that doubles agricultural production while conserving resources.\n* **Technological Advancement and Global Impact:** He showcases India\'s

#### Refine Chain Summarization

In [16]:
refine_summary_chain = load_summarize_chain(llm,
                                        chain_type='refine',
                                        verbose=True)
output = refine_summary_chain.run(text_splitter)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many